In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import arviz as az
import stan

from sklearn.linear_model import BayesianRidge
from sklearn.utils import shuffle

import nest_asyncio
nest_asyncio.apply()

# Bayesian Multiple Regression: an Insurance Model

In this notebook, we implement a multiple Bayesian Linear Regression model. This model is used to fit the [Insurance Cost dataset](https://github.com/stedy/Machine-Learning-with-R-datasets/blob/master/insurance.csv), where we predict the insurance charges based upon a patient's BMI, age, and children count.

This model is implemented in two ways: both in `Stan` and with `scikit-learn`'s Bayesian Ridge regression implementation.

## Part 0: Data Exploration

In [5]:
data = pd.read_csv('./data/insurance.csv', delimiter=";")

In [9]:
data.shape

(1338, 7)

In [15]:
# Perform a train-test split
data = shuffle(data)
train_data = data[0:1000]
test_data  = data[1000:]

(338, 7)

## Part 1: Generative Model

Next, we will build a simple Bayesian regression model of the form:
$$
\begin{align}
    \nonumber \sigma &\sim \text{Inv-Gamma}(\tau_0, \tau_1)\\
    \nonumber \alpha &\sim \mathrm{Normal}(0, \sigma_\alpha)\\
    \nonumber \beta &\sim \mathrm{MultivariateNormal}(0, \sigma_\beta \mathbb{I})\\
\nonumber y_n &\sim \mathrm{Normal}(\alpha + \beta\,x_n^T, \sigma) \quad \text{for} \,\, n = 1,\dots,N
\end{align}
$$

In [8]:
insurance_code = """ 
data {
    int<lower=1> N;
    vector[N] x;
    vector[N] y;
}

parameters {
    real alpha;
    real beta;
    real<lower=0> sigma2;
}

transformed parameters {
    real sigma;
    sigma = sqrt(sigma2);
}

model {
    // Priors
    alpha ~ normal(0, 10);
    beta ~ normal(0, 10);
    sigma2 ~ inv_gamma(1, 1);

    // Likelihood
    y ~ normal(alpha + beta * x, sigma2);
}
"""